In [40]:
# Step 1: Load the datasets

# Update with the correct path
file_path = r'C:\Users\91790\Downloads'

# Load the datasets using the file path
customers = pd.read_csv(file_path + r'\Customers.csv')
transactions = pd.read_csv(file_path + r'\Transactions.csv')

# Check the first few rows of data to ensure they are loaded correctly
print(customers.head())
print(transactions.head())


  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue   Price  
0      300.68  300.68  
1      300.68  300.68  
2      300.68  300.68  
3      601.36  300.68  
4      902.04  300.68  


In [23]:
# Merge customers and transactions on 'CustomerID'
merged_data = transactions.merge(customers, on='CustomerID', how='inner')
merged_data = merged_data.merge(products, on='ProductID', how='inner')


In [34]:
customer_features = merged_data.groupby('CustomerID').agg(
    total_purchase_value=pd.NamedAgg(column='TotalValue', aggfunc='sum'),
    transaction_count=pd.NamedAgg(column='TransactionID', aggfunc='count')
)

# Add features related to product category preferences
category_purchase = merged_data.groupby(['CustomerID', 'Category']).agg(
    category_value=pd.NamedAgg(column='TotalValue', aggfunc='sum')
).unstack(fill_value=0)

# Flatten multi-level column names from the 'unstack' operation
category_purchase.columns = [f'Category_{col}' for col in category_purchase.columns]

# Combine all features into a single DataFrame
customer_features = pd.concat([customer_features, category_purchase], axis=1)


In [33]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
# Standardizing the features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features)


In [35]:
# Compute cosine similarity between customers
similarity_matrix = cosine_similarity(scaled_features)


In [38]:
lookalike_df = pd.DataFrame.from_dict(lookalike_model, orient='index')

# Flatten the DataFrame to have CustomerID1, SimilarityScore1, CustomerID2, SimilarityScore2, CustomerID3, SimilarityScore3 columns
lookalike_df_flat = pd.DataFrame([
    (customer, *lookalike_model[customer][0], *lookalike_model[customer][1], *lookalike_model[customer][2])
    for customer in lookalike_model
], columns=['CustomerID', 'CustomerID1', 'SimilarityScore1', 'CustomerID2', 'SimilarityScore2', 'CustomerID3', 'SimilarityScore3'])

# Save the results to a CSV file
lookalike_df_flat.to_csv('Lookalike.csv', index=False)

print("Lookalike model saved successfully!")

Lookalike model saved successfully!
